In [1]:
# Instalar librerias
#!pip install pyTelegramBotAPI
#!pip install mysql.connector
#!pip install configparser
#!pip install sentiment-analysis-spanish
#!pip install keras tensorflow

In [2]:
# Importar librerias necesarias
import telebot 
import mysql.connector 
import configparser
import pytz
from datetime import datetime, timezone, timedelta
from sentiment_analysis_spanish import sentiment_analysis 

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# Crea un objeto ConfigParser y lee el archivo conexiones.properties
config = configparser.ConfigParser()
config.read('conexiones.properties')
# Accede a los valores de contexto
TELEGRAM_TOKEN = config.get('DEFAULT', 'TELEGRAM_TOKEN')
DB_HOST = config.get('DEFAULT', 'DB_HOST')
DB_USER = config.get('DEFAULT', 'DB_USER')
DB_PASS = config.get('DEFAULT', 'DB_PASS')
DB_NAME = config.get('DEFAULT', 'DB_NAME')
TIME_ZONE = config.get('DEFAULT', 'TIME_ZONE')
# Definir zona horaria
tz = pytz.timezone(TIME_ZONE) 

In [ ]:
# Conectarse a la base de datos
def load_message_db(id_mensaje, id_chat, texto, usuario, fecha_hora, clasificacion):
    # crear conector
    mydb = mysql.connector.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASS,
        database=DB_NAME
    )
    # Crear un cursor para ejecutar consultas
    cursor = mydb.cursor()
    # Hacer insert
    sql = '''INSERT INTO mensajes_texto 
    (id_mensaje, id_chat, texto, usuario, fecha_hora, clasificacion) 
    VALUES (%s, %s, %s, %s, %s, %s)'''
    # Tupla de valores para la consulta
    values = (id_mensaje, id_chat, texto, usuario, fecha_hora, clasificacion)
    # Ejecutar la consulta con los valores como parámetros
    cursor.execute(sql, values)
    mydb.commit()
    # Cerrar conexión
    cursor.close()
    mydb.close()

In [ ]:
# Crea clasificador
sentiment = sentiment_analysis.SentimentAnalysisSpanish()

In [ ]:
def analize_text(texto):
    texto = texto.strip()
    if len(texto) < 5:
        return 0
    analisis = sentiment.sentiment(texto)
    analisis = 1 - analisis
    analisis = float(analisis)
    #clasif = "neutro"
    #if(analisis >= 0.7):
    #    clasif = "negativo"
    #if(analisis <= 0.3):
    #    clasif = "positivo"
    return analisis

In [ ]:
# Crea la instancia del bot
bot_token = TELEGRAM_TOKEN
bot = telebot.TeleBot(bot_token)

In [ ]:
@bot.message_handler(commands=['start'])
def handle_start(message):
    bot.reply_to(message, "¡Hola! Bienvenido al bot de pruebas.")

In [ ]:
# Manejador de mensajes
@bot.message_handler(func=lambda message: True)
def handle_messages(message):
    # Extraer id, grupo, texto, usuario y fecha
    message_id = message.message_id
    chat_id = message.chat.id 
    text = message.text
    fecha_hora_actual = datetime.now(tz)
    date = fecha_hora_actual.strftime('%Y-%m-%d %H:%M:%S')
    user = message.from_user
    username = user.username
    # Verificar si el nombre de usuario está presente
    if username:
        user_info = username
    else:
        first_name = user.first_name
        last_name = user.last_name if user.last_name else ""
        user_info = f"{first_name}{last_name}" if (first_name or last_name) else "Sin Nombre"
    user_info = user_info.strip()
    # Clasificar mensaje
    clasificacion = analize_text(text)
    load_message_db(message_id, chat_id, text, user_info, date, clasificacion)

In [ ]:
# Inicia el bot
bot.polling(none_stop=True)